In [1]:
from PySide2.QtWidgets import *
from PySide2.QtCore import *
from PySide2.QtGui import *
from PySide2.QtMultimedia import *
from PySide2.QtMultimediaWidgets import *
import os
import cv2
import sys
import numpy as np

#from deeplabcut.utils import auxiliaryfunctions
from PySide2.QtCore import QObject, Signal     

from matplotlib.backends.backend_qt5agg import (FigureCanvas)
from matplotlib.figure import Figure

from pathlib import Path, PurePath

In [66]:
%gui qt
class FrameExtractor(QMainWindow):
    frameIndexChanged = Signal()

    def __init__(self, *, config=None):
        super().__init__()

        self.setWindowTitle("Frame Extractor")

        #TODO: size of canvas
        
        self.mainWidget = QWidget()
        self.setCentralWidget(self.mainWidget)
        layout = QVBoxLayout(self.mainWidget)
        
        self.canvas = FigureCanvas(Figure())
        layout.addWidget(self.canvas)
        #self.addToolBar(QtCore.Qt.BottomToolBarArea,
          #              NavigationToolbar(dynamic_canvas, self))

        self.ax = self.canvas.figure.subplots(1, 1)
        self.ax.set_visible(False)
        
        closeShortcut = QShortcut(QKeySequence(self.tr("Ctrl+Q")), self)
        closeShortcut.activated.connect(self.close)

        self.frameIndexChanged.connect(self.updateCanvas)

        self.statusBar = QStatusBar()
        self.setStatusBar(self.statusBar)
        
        ################################################################
        # Add widgets with the following convention:
        # 1. Instantiate
        # 2. Set any shortcuts
        # 3. Set any attributes e.g. tool tips
        # 4. Connections
        # 5. Enable or disable
        # 6. Add to layout
        
        self.openButton = QPushButton(self.tr("Open Video File"))
        self.openButton.setShortcut(QKeySequence(self.tr("Ctrl+O")))
        self.openButton.clicked.connect(self.loadVideo)
        self.openButton.setEnabled(True)
        layout.addWidget(self.openButton)

        self.closeButton = QPushButton(self.tr("Close Video File"))
        self.closeButton.setShortcut(QKeySequence(self.tr("Ctrl+W")))
        self.closeButton.clicked.connect(self.closeVideo)
        self.closeButton.setEnabled(False)
        layout.addWidget(self.closeButton)
        
        self.previousButton = QPushButton(self.tr("Previous Frame"))
        self.previousButton.setShortcut(QKeySequence(self.tr("Left")))
        self.previousButton.clicked.connect(self.previousFrame)
        self.previousButton.setEnabled(False)
        layout.addWidget(self.previousButton)

        self.nextButton = QPushButton(self.tr("Next Frame"))
        self.nextButton.setShortcut(QKeySequence(self.tr("Right")))
        self.nextButton.clicked.connect(self.nextFrame)
        self.nextButton.setEnabled(False)
        layout.addWidget(self.nextButton)
        
        self.playbackButton = QPushButton(self.tr("Play Video"))
        # Not sure why this doesn't always work
        self.playbackButton.setShortcut(QKeySequence(self.tr("Space")))
        self.playbackButton.clicked.connect(self.playback)
        self.playbackButton.setEnabled(False)
        layout.addWidget(self.playbackButton)
        
        self.numCaptureLabel = QLabel()
        self.numCaptureLabel.setText(self.tr("Number of frames to capture"))
        layout.addWidget(self.numCaptureLabel)

        self.numCaptureWidget = QSpinBox()
        self.numCaptureWidget.setValue(1)
        self.numCaptureWidget.valueChanged.connect(self.updateNumCapture)
        self.numCaptureWidget.setEnabled(False)
        layout.addWidget(self.numCaptureWidget)
        
        self.scrollStepLabel = QLabel()
        self.scrollStepLabel.setText(self.tr("Scroll Step"))
        self.scrollStepLabel.setToolTip(self.tr("Number of frames to increment/decrement by"
                                               " every time you scroll with your mouse"))
        layout.addWidget(self.scrollStepLabel)
        
        self.scrollStepWidget = QSpinBox()
        self.scrollStepWidget.setValue(1)
        self.scrollStepWidget.valueChanged.connect(self.changeScrollStep)
        self.scrollStepWidget.setEnabled(False)
        layout.addWidget(self.scrollStepWidget)
   
        self.slider = QSlider()
        self.slider.setOrientation(Qt.Horizontal)
        self.slider.valueChanged.connect(self.sliderChanged)
        self.slider.setEnabled(False)
        layout.addWidget(self.slider)
        
        self.reset()

    def updateCanvas(self):

        # make sure within range
        self.frameIndex = min(self.frameIndex, self.numFrames -1)
        self.frameIndex = max(self.frameIndex, 0)

        # previous and next buttons are disabled during playback, 
        # no need to check
        if not self.isVideoPlaying:
            self.checkPreviousNextButtons()

        self.vid.set(1, self.frameIndex)
        ret, frame = self.vid.read()
        if ret:
            self.slider.setValue(self.frameIndex)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            if self.frameIndex == 0:
                self.img = self.ax.imshow(frame)
            else:
                # For speed; only need to call imshow once
                self.img.set_data(frame)

            self.ax.set_title("Frame {} out of {}".format(self.frameIndex, self.numFrames - 1))
#             self.ax.set_xticks([])
#             self.ax.set_yticks([])
#             self.axes.set_title(str(str(self.currFrame)+"/"+str(self.numberFrames-1) +" "+ self.filename))
            self.ax.figure.canvas.draw()
    
            if self.isVideoPlaying:
                self.frameIndex += 1
        else:
            if self.isVideoPlaying:
                if self.timer.isActive():
                    self.timer.stop()

    def loadVideo(self):

        videoPath, _ = QFileDialog.getOpenFileName(self, 'Load Video',
                                             '/home/jchu/Downloads')
        videoPath = Path(videoPath)

        # If user simply quits dialog, videoPath is empty.
        # Only proceed if videoPath is a valid file
        if videoPath.is_file():
            vid = cv2.VideoCapture(str(videoPath))
            # Check whether video is valid
            if not vid.isOpened():
                msg = QMessageBox(self)
                msg.setWindowTitle("Error")
                msg.setText("Could not load video. Do you want to retry?")
                msg.setIcon(QMessageBox.Critical)
                msg.addButton(QMessageBox.Yes)
                msg.addButton(QMessageBox.No)
                msg.setDefaultButton(QMessageBox.Yes)

                if msg.exec_() == QMessageBox.Yes:
                    self.loadVideo()

            else:
                numFrames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))

                if numFrames < 2:  # images
                    msg = QMessageBox(self)
                    msg.setWindowTitle("Error")
                    msg.setText("File {} is not a video. Do you want to retry?".format(videoPath.name))
                    msg.setIcon(QMessageBox.Critical)
                    msg.addButton(QMessageBox.Yes)
                    msg.addButton(QMessageBox.No)
                    msg.setDefaultButton(QMessageBox.Yes)

                    if msg.exec_() == QMessageBox.Yes:
                        self.loadVideo()

                else:
                    # If we got to this point, we know it's a valid video. Enable controls
                    self.openButton.setEnabled(False)
                    self.closeButton.setEnabled(True)
                    self.previousButton.setEnabled(False)
                    self.nextButton.setEnabled(True)
                    self.playbackButton.setEnabled(True)

                    self.numCaptureWidget.setMinimum(1)
                    self.numCaptureWidget.setMaximum(numFrames)
                    self.numCaptureWidget.setValue(1)
                    self.numCaptureWidget.setEnabled(True)
                    
                    self.scrollStepWidget.setMinimum(1)
                    self.scrollStepWidget.setMaximum(numFrames - 1)
                    self.scrollStepWidget.setValue(1)
                    self.scrollStepWidget.setEnabled(True)
                    
                    self.slider.setValue(0)
                    self.slider.setMinimum(0)
                    self.slider.setMaximum(numFrames - 1)
                    self.slider.setEnabled(True)

                    self.frameIndex = 0
                    self.numCapture = 1
                    self.numFrames = numFrames
                    self.isVideoLoaded = True
                    self.isVideoPlaying = False
                    self.vid = vid
                    self.videoPath = videoPath
                    self.img = None

                    self.statusBar.showMessage("Working on video '{}'".format(videoPath.name))
                    self.ax.set_visible(True)

                    self.frameIndexChanged.emit()

    def closeVideo(self):
        
        self.reset()        
        
    def previousFrame(self):
        
        self.frameIndex -= 1
        self.frameIndexChanged.emit()
        
    def nextFrame(self):

        self.frameIndex += 1
        self.frameIndexChanged.emit()

    def playback(self):

        if self.isVideoPlaying:
            self.isVideoPlaying = False
            self.playbackButton.setText(self.tr("Play Video"))
            self.openButton.setEnabled(False)
            self.closeButton.setEnabled(True)
            self.numCaptureWidget.setEnabled(True)
            self.scrollStepWidget.setEnabled(True)
            self.slider.setEnabled(True)
            
            self.checkPreviousNextButtons()
            self.timer.stop()
            
        else:
            self.isVideoPlaying = True
            self.playbackButton.setText(self.tr("Pause Video"))
            self.openButton.setEnabled(False)
            self.closeButton.setEnabled(False)
            self.previousButton.setEnabled(False)
            self.nextButton.setEnabled(False)
            self.numCaptureWidget.setEnabled(False)
            self.scrollStepWidget.setEnabled(False)
            self.slider.setEnabled(False)

            fps = self.vid.get(cv2.CAP_PROP_FPS)
            self.timer = QTimer()
            self.timer.setInterval(int(np.floor(1/fps*1000)))
            self.timer.timeout.connect(self.updateCanvas)
            self.timer.start()
            
    def updateNumCapture(self):
        
        self.numCapture = self.numCaptureWidget.value()

    def wheelEvent(self, event):
        
        # only scroll through frames if a valid video
        # is loaded
        if self.isVideoLoaded:
            if np.sign(event.angleDelta().y()) == 1:
                self.frameIndex += self.scrollStep
            else:
                self.frameIndex -= self.scrollStep
            self.frameIndexChanged.emit()
        else:
            pass

    def changeScrollStep(self):

        try:
            step = max(int(self.scrollStepWidget.value()), 1)
            step = min(step, self.numFrames)
            self.scrollStepWidget.setValue(step)
            self.scrollStep = step
        except:
            pass

    def sliderChanged(self):
        
        self.frameIndex = self.slider.value()
        self.frameIndexChanged.emit()
        
    def checkPreviousNextButtons(self):
        
        if self.frameIndex == 0:
            self.previousButton.setEnabled(False)
        else:
            self.previousButton.setEnabled(True)
            
        if self.frameIndex < self.numFrames - 1:
            self.nextButton.setEnabled(True)
        else:
            self.nextButton.setEnabled(False)
            
    def reset(self):
        self.openButton.setEnabled(True)
        self.closeButton.setEnabled(False)
        self.previousButton.setEnabled(False)
        self.nextButton.setEnabled(False)
        self.playbackButton.setEnabled(False)
        self.numCaptureWidget.setEnabled(False)
        self.scrollStepWidget.setEnabled(False)
        self.slider.setEnabled(False)

        self.frameIndex = 0

        self.numFrames = 0
        self.numCapture = 1
        self.scrollStep = 1
        self.isVideoLoaded = False
        self.isVideoPlaying = False
        self.vid = None
        self.videoPath = None
        self.img = None
        self.timer = QTimer()

        self.statusBar.showMessage("")
        self.ax.cla()
        self.ax.set_visible(False)
        self.ax.figure.canvas.draw()

    def closeEvent(self, event):
        msg = QMessageBox(self)
        msg.setWindowTitle("Close Window")
        msg.setText("Are you sure you want to exit?")
        msg.setIcon(QMessageBox.Question)
        msg.addButton(QMessageBox.Yes)
        msg.addButton(QMessageBox.No)
        msg.setDefaultButton(QMessageBox.Yes)

        if msg.exec_() == QMessageBox.Yes:
            app = QApplication.instance()
            super().closeEvent(event)
            app.exit()
        else:
            event.ignore()

app = QApplication.instance()
if app is None:
    app = QApplication([])
w = FrameExtractor()
w.show()
app.exec_()

0